# 0. Setup Paths
This code block is dedicated to setting up the environment for our project. We're defining some important variables and also creating directories in our workspace. Here's a brief overview:

1. **Model Selection:** We select the pre-trained model we want to fine-tune. In this case, we are using the SSD MobileNet V2 FPNLite 320x320 model, which is a fast and efficient object detection model. We store the model's name and download URL for later use.
2. **File Names:** We set the names of some files that we will use later, such as the script to generate TensorFlow records and the label map file.
3. **Paths:** We create a dictionary to store the paths to various directories we'll use throughout our project. This includes the workspace, scripts, models, annotations, images, and more. We'll use these paths to read and write data as we proceed.
4. **File Paths:** Similarly, we create a dictionary for file paths that we will use throughout the project.
5. **Directory Creation:** Finally, we iterate through all the path values in our paths dictionary. If any of these directories do not already exist, we create them using the mkdir command. This ensures we have a well-structured workspace before we begin our project.

By organizing our workspace in this manner, we ensure that we have a neat and systematic setup, making it easy to manage our project as it becomes more complex.

In [ ]:
import os

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

This code performs several operations needed for the setup of the TensorFlow Object Detection API. It starts by installing necessary dependencies based on the operating system type (Windows or Linux). Then, it clones the TensorFlow Object Detection API from the official repository if it's not already installed. For Linux, it installs protobuf-compiler and sets up the Object Detection API, while for Windows, it downloads protobuf, sets it up along with

In [ ]:
# https://www.tensorflow.org/install/source_windows

# Depending on the operating system, the installation and setup process varies.
# For a Windows system:
if os.name=='nt':
    # Install wget python package
    !pip install wget
    import wget

# Checking if the TensorFlow Object Detection API is already installed. If not, clone it from the official GitHub repository.
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

# If the operating system is Linux, install protobuf-compiler and setup the TensorFlow Object Detection API.
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python3 -m pip install .

# If the operating system is Windows, download protobuf, setup it and the TensorFlow Object Detection API.
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e .

# Run a script provided by TensorFlow to verify the installation.
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
!python {VERIFICATION_SCRIPT}

# Upgrade tensorflow
!pip install tensorflow --upgrade

# Uninstall and re-install protobuf and matplotlib due to version conflicts
!pip uninstall protobuf matplotlib -y
!pip install protobuf matplotlib==3.2

# Import the object detection module
import object_detection

# Print a list of installed python packages for verification
!pip list

# Download and extract the pre-trained model from TensorFlow model zoo based on the type of operating system.
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

# 2. Create Label Map
This section of code is creating a label map for the custom object detection model. The label map tells the model what each object is by assigning an ID to a specific object. In this case, we are telling the model that the object with ID 1 is a 'Nut' and the object with ID 2 is a 'Screw'.

The label map is saved in the 'protobuf' (.pbtxt) format, which is the required format for TensorFlow's Object Detection API. Each item in the label map is defined by its name and ID. This label map file will later be used in the training and detection stages of the object detection model.

In [ ]:
labels = [{'name':'Nut', 'id':1}, {'name':'Screw', 'id':2}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records
First, it checks if there is a tarball archive (.tar.gz) of images present in the defined IMAGE_PATH. If such an archive exists, it uncompresses the archive using the tar command. The -zxvf flag tells tar to extract the files (-x), be verbose and print the file names as they are extracted (-v), use GZIP to uncompress (-z), and read from the specified file (-f).

Then, it clones a Git repository that contains a script to generate TensorFlow records (TFRecords). TFRecords are a binary file format for storing data, and TensorFlow's preferred format for storing large amounts of data like images. The repository is cloned into the previously defined SCRIPTS_PATH.

Finally, it runs the generate_tfrecord.py script. This script converts the image and corresponding annotation data into the TFRecord format. The script takes a few arguments:

- -x or --xml_dir: The path to the directory containing the xml files with the bounding box annotations. In this case, it is the train directory inside IMAGE_PATH.
- -l or --labels_path: The path to the label_map.pbtxt file, which was generated in a previous step.
- -o or --output_path: The path where the generated TFRecord file will be saved. Here, it's the train.record file inside ANNOTATION_PATH.
After this script runs, the training data will be in the format expected by TensorFlow's Object Detection API and ready to use for training the object detection model.

In [ ]:
# OPTIONAL IF RUNNING ON COLAB
ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
if os.path.exists(ARCHIVE_FILES):
  !tar -zxvf {ARCHIVE_FILES}

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

# 4. Copy Model Config to Training Folder
This copies the pipeline.config file from the pre-trained model's directory to your model's training directory.

In the context of TensorFlow's Object Detection API, the pipeline.config file is a configuration file that defines various parameters for the object detection task such as the model architecture, the path for the train and validation data, the path for the label map, the number of classes, the batch size, the number of steps, learning rate, etc.

This configuration file serves as the recipe for training a specific type of model. The original pipeline.config file comes from the pretrained model's directory, which means it was configured for that specific model. By copying it into your training directory, you ensure that your model will be trained with the same configuration. However, you may need to update some paths and parameters in the configuration file to fit your specific use case.

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# 5. Update Config For Transfer Learning
This section of the code is related to loading and modifying the pipeline.config file for training your own model. The pipeline.config file is read, updated and then written back to the disk.

Here is the detailed explanation of each part:

1. TensorFlow and various utilities from the Object Detection API are imported, including config_util for handling configuration files, pipeline_pb2 for protocol buffer message types, and Google's protobuf library for working with protocol buffers.

2. The current pipeline configuration is read using the get_configs_from_pipeline_file function, which takes the path of pipeline.config file and returns a dictionary of configurations.

3. A new TrainEvalPipelineConfig object is created. This object will be populated with the configurations from the pipeline.config file.

4. The pipeline.config file is opened and its contents are read into a string.

5. The configuration string is merged into the TrainEvalPipelineConfig object using the Merge function from text_format.

6. The test record path in the configuration object is updated with the actual path of your test record file.

7. The updated configuration object is converted back to a string using MessageToString.

8. The updated configuration string is written back to the pipeline.config file.

The updated pipeline.config file now contains the correct paths for the test record file, and can be used to train your model.

In [ ]:
# Import necessary libraries
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

# Load the pipeline.config file into a config object
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

# Create a new TrainEvalPipelineConfig object
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

# Read the pipeline.config file and merge it into the TrainEvalPipelineConfig object
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

# Update the test record file path in the config object
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

# Convert the updated config object back into a string
config_text = text_format.MessageToString(pipeline_config)

# Write the updated config string back to the pipeline.config file
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

# 6. Train the model
In this section of the code, we are setting up and running the model training process:

1. The script to train the model, model_main_tf2.py, is located within the TensorFlow's object_detection module. A path to this script is constructed.

2. A command is constructed to run the training script using Python. This command includes the following options:

- --model_dir: This is the path where the model checkpoints will be saved during training.
- --pipeline_config_path: This is the path to the pipeline.config file, which was prepared and updated in the previous steps.
- --num_train_steps: This sets the number of steps for which to train the model. In this case, it's set to 2000, but this can be increased if you have more data or want the model to learn for a longer period.
3. The constructed command is printed, and then run using the ! operator, which is specific to Jupyter notebooks and is used to run shell commands.

In [ ]:
# Define the path to the training script
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

# Create the command to run the training script
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

# Print the command
print(command)

# Execute the command
!{command}

This section of the code sets up and executes the model training process. The training script, model_main_tf2.py, is located within the TensorFlow's object_detection module. A command is then constructed to run this script with specific options indicating where to save the model checkpoints (--model_dir), where to find the pipeline.config file (--pipeline_config_path), and how many steps to train the model for (--num_train_steps). This command is then executed in the shell using the ! operator.

# 7. Evaluate the Model
In this section of the code, we are preparing the command to run the model evaluation process:

A command is constructed to run the same training script as before, model_main_tf2.py, but with different options. This command includes the following options:
- --model_dir: This is the path where the model checkpoints were saved during training.
- --pipeline_config_path: This is the path to the pipeline.config file.
- --checkpoint_dir: This is the path where the model checkpoints are stored, and from where the model to be evaluated will be loaded. In this case, it's the same as model_dir.

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}

This section of the code prepares the command to run the model evaluation process. The same training script, model_main_tf2.py, is used, but with different options. The --model_dir option specifies the path to the model checkpoints, the --pipeline_config_path option specifies the path to the pipeline.config file, and the --checkpoint_dir option specifies the path where the model checkpoints are stored and from where the model to be evaluated will be loaded. The constructed command is stored in the variable command for later execution.

# 8. Load Train Model From Checkpoint
In this section of the code, we are loading the trained model checkpoint and defining a function for object detection:

1. The TensorFlow and object detection libraries are imported.
2. The pipeline configuration is loaded using config_util.get_configs_from_pipeline_file, which retrieves the configurations from the pipeline.config file.
3. The detection model is built using model_builder.build, with the model configuration obtained from the pipeline configurations.
4. The model checkpoint is restored using tf.compat.v2.train.Checkpoint and the restore function. The checkpoint file is specified as the ckpt-11 file in the checkpoint directory.
5. A detect_fn function is defined. This function takes an image as input, preprocesses it using the detection model's preprocess function, performs prediction using the predict function, and then postprocesses the predictions using the postprocess function. The resulting detections are returned.

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-11')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In this section of the code, we load the trained model checkpoint and define a function for object detection. First, the necessary libraries are imported. Then, the pipeline configuration is loaded using config_util.get_configs_from_pipeline_file. The detection model is built using model_builder.build with the model configuration from the pipeline. The model checkpoint is restored using tf.compat.v2.train.Checkpoint and restore. Finally, the detect_fn function is defined, which takes an image as input, preprocesses it using the detection model's preprocess function, performs prediction using the predict function, and postprocesses the predictions using the postprocess function. The resulting detections are returned.

# 9. Detect from an Image
In this final section of the code, we perform object detection on a test image using the loaded model:

1. The OpenCV, NumPy, and Matplotlib libraries are imported.
2. The category index is created using label_map_util.create_category_index_from_labelmap based on the label map file.
3. The test image path is specified and read using cv2.imread.
4. The image is converted to a NumPy array.
5. The input tensor is created by converting the image array to a TensorFlow tensor.
6. The detect_fn function is called with the input tensor to obtain the detections.
7. The number of detections is extracted and the detections dictionary is updated accordingly.
8. The detection classes are converted to integers and the label ID offset is applied.
9. A copy of the image is created for visualization purposes.
10. The visualize_boxes_and_labels_on_image_array function from viz_utils is called to draw bounding boxes and labels on the image.
11. The resulting image with detections is displayed using plt.imshow and plt.show().

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib tk

# Create category index
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

# Load and preprocess the test image
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'jorge6.jpg')
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

# Convert image to TensorFlow tensor
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

# Perform object detection
detections = detect_fn(input_tensor)

# Process detection results
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detections['num_detections'] = num_detections

# Convert detection classes to integers and apply label ID offset
label_id_offset = 1
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

# Create a copy of the image for visualization
image_np_with_detections = image_np.copy()

# Draw bounding boxes and labels on the image
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    detections['detection_classes'] + label_id_offset,
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=100,
    min_score_thresh=0.26,
    agnostic_mode=False
)

# Display the image with detections
plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

In this final section of the code, object detection is performed on a test image using the loaded model. The necessary libraries are imported, including OpenCV, NumPy, and Matplotlib. The category index is created from the label map file. The test image is loaded and preprocessed. The image is converted to a TensorFlow tensor and passed through the detect_fn function to obtain the detections. The detection results are processed and visualized by drawing bounding boxes and labels on a copy of the image. Finally, the image with detections is displayed using Matplotlib.

# 10. Real Time Detections from your Webcam

In [ ]:
!pip uninstall opencv-python-headless -y

In [ ]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break